In [1]:
!nvidia-smi

Sat Apr 27 11:57:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q pypdf python-dotenv

In [3]:
!pip -q install git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install -q datasets loralib sentencepiece

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-huggingface 0.1.4 requires huggingface-hub<0.21.0,>=0.20.3, but you have huggingface-hub 0.22.2 which is incompatible.


In [5]:
!pip install -q einops accelerate langchain bitsandbytes

In [6]:
!pip install sentence_transformers

In [7]:
!pip install llama-index

In [8]:
!pip install -q chromadb

In [9]:
!pip install llama-index-llms-huggingface

  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.0 requires huggingface-hub>=0.21.2, but you have huggingface-hub 0.20.3 which is incompatible.


In [10]:
!pip install llama-index-embeddings-huggingface


In [11]:
!pip install llama-index-vector-stores-chroma

In [1]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, VectorStoreIndex

from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.vector_stores.chroma import ChromaVectorStore


import chromadb

from IPython.display import Markdown, display

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [3]:
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection('climate_report')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/OncoAIFlow").load_data()

In [6]:
from llama_index.core.prompts.prompts import SimpleInputPrompt
system_prompt = "Tu es un assistant medical. Tu dois repondre uniquement par oui ou par non. si tu n'est pas sur de toi, alors reponds par oui."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [7]:
from huggingface_hub import login
login()

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(ma

In [9]:
!huggingface-cli whoami

Acsts


In [10]:
import torch
torch.set_default_device('cuda')

In [14]:
llm = HuggingFaceLLM(
    context_window=8000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    tokenizer_kwargs={"max_length": 8000},
    model_kwargs={"torch_dtype": torch.float16}

)

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 23.06 MiB is free. Process 186237 has 14.72 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 139.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
embed_model =  HuggingFaceEmbedding(model_name="thenlper/gte-base")

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

In [ ]:
index

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Est-ce que le traitement systemique fait partie des traitements possibles pour un patient un performance status a 4, porteur d'un CHC unifocal, avec une maladie hepatique CHILD A")
display(Markdown(f"<b>{response}</b>"))

In [ ]:
import random

# Liste de valeurs pour les balises
performance = ['0', '1', '2', '3', '4']
traitement = ['la transplantation', 'la resection', 'la destruction percutanee', 'la radiotherapie', 'le traitement systemique', 'la chimio-embolisation', 'la radiotherapie interne selective']
nodule = ['un nodule', 'deux nodules', 'trois nodules', 'de multiples nodules']
maladie_hepatique = ['sur foie non cirrhotique','avec cirrhose CHILD A', 'avec cirrhose CHILD B', 'avec cirrhose CHILD C']

# Fonction pour générer un exemple aléatoire
def generer_prompt():
    performance_aleatoire = random.choice(performance)
    traitement_aleatoire = random.choice(traitement)
    nodule_aleatoire = random.choice(nodule)
    maladie_hepatique_aleatoire = random.choice(maladie_hepatique)
    return f"Est-ce que {traitement_aleatoire} fait partie des traitements envisageable pour un patient un performance status a {performance_aleatoire}, porteur d'un CHC comportant {nodule_aleatoire}, {maladie_hepatique_aleatoire}."

# Exemple d'utilisation
for _ in range(5):
    print(generer_prompt())


Est-ce que la destruction percutanee fait partie des traitements envisageable pour un patient un performance status a 3, porteur d'un CHC comportant de multiples nodules, avec cirrhose CHILD A.
Est-ce que la resection fait partie des traitements envisageable pour un patient un performance status a 0, porteur d'un CHC comportant un nodule, avec cirrhose CHILD B.
Est-ce que le traitement systemique fait partie des traitements envisageable pour un patient un performance status a 3, porteur d'un CHC comportant deux nodules, avec cirrhose CHILD A.
Est-ce que le traitement systemique fait partie des traitements envisageable pour un patient un performance status a 1, porteur d'un CHC comportant deux nodules, avec cirrhose CHILD B.
Est-ce que la resection fait partie des traitements envisageable pour un patient un performance status a 4, porteur d'un CHC comportant de multiples nodules, sur foie non cirrhotique.


In [ ]:
response = query_engine.query("Est-ce que la resection chirurgicale fait partie des traitements possibles pour un patient un performance status a 1, porteur d'un CHC unifocal, avec une maladie hepatique CHILD A")
display(Markdown(f"<b>{response}</b>"))

In [ ]:
for _ in range(5):
    query_engine = index.as_query_engine()
    prompt = generer_prompt()
    print(prompt)
    response = query_engine.query(prompt)

    display(Markdown(f"<b>{response}</b>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que le traitement systemique fait partie des traitements envisageable pour un patient un performance status a 2, porteur d'un CHC comportant deux nodules, sur foie non cirrhotique.


<b>Oui.</b>

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que la radiotherapie interne selective fait partie des traitements envisageable pour un patient un performance status a 4, porteur d'un CHC comportant deux nodules, sur foie non cirrhotique.


...

Est-ce que la destruction percutanee fait partie des traitements envisageable pour un patient un performance status a 4, porteur d'un CHC comportant un nodule, avec cirrhose CHILD C.


<b>Non.</b>

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que la transplantation fait partie des traitements envisageable pour un patient un performance status a 0, porteur d'un CHC comportant deux nodules, sur foie non cirrhotique.


<b>Oui.</b>